In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("HM1").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

23/11/02 12:18:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
#!pip3 install pandas

In [3]:
import pandas as pd
from datetime import datetime

#parse the date format and the csv file into pandas DataFrame

custom_date_parser = lambda x: datetime.strptime(x, "%B %d, %Y")

read_data = pd.read_csv(r"data/spacenews-202309.csv", parse_dates=['date'], date_parser=custom_date_parser)
read_data

/tmp/ipykernel_41/3838846475.py:8: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  read_data = pd.read_csv(r"data/spacenews-202309.csv", parse_dates=['date'], date_parser=custom_date_parser)


,title,url,content,author,date,postexcerpt
0,HawkEye 360 reaches inflection point on path t...,https://spacenews.com/hawkeye-360-reaches-infl...,PARIS — With a recent funding round and growin...,Jeff Foust,2023-09-14,NaN
1,SES Q&A | Leveling up multi-orbit connectivity,https://spacenews.com/ses-qa-leveling-up-multi...,Multi-orbit satellite operator SES is on the v...,Jason Rainbow,2023-09-13,NaN
2,Rapid Starlink iteration poses challenges for ...,https://spacenews.com/rapid-starlink-iteration...,"TAMPA, Fla. — SpaceX’s ability to quickly chan...",Jason Rainbow,2023-09-13,NaN
3,Space Force to release guidelines for the use ...,https://spacenews.com/space-force-to-release-g...,WASHINGTON — U.S. Chief of Space Operations Ge...,Sandra Erwin,2023-09-13,NaN
4,ULA has ‘no issues’ with Space Force plan to s...,https://spacenews.com/ula-has-no-issues-with-s...,"WASHINGTON — United Launch Alliance, one of ju...",Sandra Erwin,2023-09-13,NaN
...,...,...,...,...,...,...
19579,Kendall lays out Pentagon thinking on future s...,https://spacenews.com/frank-kendall-at-wsbr/,"\nFrank Kendall, the Pentagon’s top acquisitio...",SpaceNews Staff,2016-02-25,"Frank Kendall, the Pentagon’s top acquisition ..."
19580,A larger share of NOAA’s declining space budge...,https://spacenews.com/a-larger-share-of-noaas-...,Updated Feb. 10 at 10:18 p.m. Eastern The U.S....,Debra Werner,2016-02-10,The U.S. National Oceanic and Atmospheric Admi...
19581,Think Tank Turns Its Attention To Mars As 2016...,https://spacenews.com/think-tank-turns-its-att...,WASHINGTON — As NASA develops a long-term stra...,Jeff Foust,2015-06-11,As NASA develops a long-term strategy to suppo...
19582,House Bill Leaves Last Three JPSS Satellites i...,https://spacenews.com/no-money-for-noaa-weathe...,WASHINGTON — A spending bill the House passed ...,Dan Leone,2015-06-04,A spending bill the House passed June 3 would ...


In [4]:
read_data.dtypes

title                  object
url                    object
content                object
author                 object
date           datetime64[ns]
postexcerpt            object
dtype: object

#### List the records that the term “Space” (uppercase and lowercase included) occurs both in ‘Title’ and ‘Postexcerpt’

In [5]:
from pyspark import SparkFiles
from pyspark.sql.types import *
news_data_schema = StructType([StructField("Title", StringType(), True)\
                               ,StructField("Url", StringType(), True)\
                               ,StructField("Content", StringType(), True)\
                               ,StructField("Author", StringType(), True)\
                               ,StructField("Date", DateType(), True)\
                               ,StructField("Postexcerpt", StringType(), True)])

news_data = spark.createDataFrame(read_data, schema = news_data_schema)
news_data.show(n=10)

23/11/02 12:18:20 WARN TaskSetManager: Stage 0 contains a task of very large size (19966 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+--------------------+--------------------+-------------+----------+-----------+
|               Title|                 Url|             Content|       Author|      Date|Postexcerpt|
+--------------------+--------------------+--------------------+-------------+----------+-----------+
|HawkEye 360 reach...|https://spacenews...|PARIS — With a re...|   Jeff Foust|2023-09-14|        NaN|
|SES Q&A | Levelin...|https://spacenews...|Multi-orbit satel...|Jason Rainbow|2023-09-13|        NaN|
|Rapid Starlink it...|https://spacenews...|TAMPA, Fla. — Spa...|Jason Rainbow|2023-09-13|        NaN|
|Space Force to re...|https://spacenews...|WASHINGTON — U.S....| Sandra Erwin|2023-09-13|        NaN|
|ULA has ‘no issue...|https://spacenews...|WASHINGTON — Unit...| Sandra Erwin|2023-09-13|        NaN|
|Microsoft and Esr...|https://spacenews...|PARIS – To speed ...| Debra Werner|2023-09-13|        NaN|
|Intelsat and Aaly...|https://spacenews...|SAN FRANCISCO – G...| Debra Werner|2023

In [6]:
#List of term "Space" records

news_data.filter((news_data.Title.rlike("(?i)^*state")) & (news_data.Postexcerpt.rlike("(?i)^*state"))).show()

23/11/02 12:18:25 WARN TaskSetManager: Stage 1 contains a task of very large size (19966 KiB). The maximum recommended task size is 1000 KiB.
23/11/02 12:18:31 WARN TaskSetManager: Stage 2 contains a task of very large size (22850 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+--------------------+--------------------+-------------+----------+--------------------+
|               Title|                 Url|             Content|       Author|      Date|         Postexcerpt|
+--------------------+--------------------+--------------------+-------------+----------+--------------------+
|‘State of the spa...|https://spacenews...|WASHINGTON — For ...| Sandra Erwin|2022-08-24|The "State of the...|
|Washington state ...|https://spacenews...|SAN FRANCISCO – W...| Debra Werner|2022-02-22|Washington state’...|
|State Fight: Shor...|https://spacenews...|Terran Orbital, w...|Jason Rainbow|2022-02-20|The Sunshine Stat...|
|State Fight: New ...|https://spacenews...|Federal space fun...| Debra Werner|2022-02-19|New Mexico was no...|
|State Fight: A co...|https://spacenews...|Cities, counties ...| Debra Werner|2022-02-17|With California l...|
|Industry offers c...|https://spacenews...|ORLANDO, Fla. — A...|   Jeff Foust|2022-01-17|After a year that...|
|

In [7]:
news_data.filter( (news_data.Title.rlike("(?i)^*state")) & (news_data.Postexcerpt.rlike("(?i)^*state")) ).count()

23/11/02 12:18:36 WARN TaskSetManager: Stage 3 contains a task of very large size (19966 KiB). The maximum recommended task size is 1000 KiB.


38

In [8]:
"""print(news_data.filter(news_data.Title.contains("State")).count(), news_data.filter(news_data.Title.contains("state")).count())"""

'print(news_data.filter(news_data.Title.contains("State")).count(), news_data.filter(news_data.Title.contains("state")).count())'

In [9]:
"""print(news_data.filter(news_data.Title.rlike("(?i)^*state")).count())
news_data.filter(news_data.Title.rlike("(?i)^*state")).show()"""

'print(news_data.filter(news_data.Title.rlike("(?i)^*state")).count())\nnews_data.filter(news_data.Title.rlike("(?i)^*state")).show()'

In [10]:
import string
import re
import pyspark.sql.functions as func
#Remove punctuations and extra white space

def remove_punctuations(text):
    text = text.lower()
    text = re.sub(r'[^A-Za-z0-9]+', ' ', text)
    text = re.sub("\s+", " ", text)
    text = text.strip()
    return text

udf_remove_punctuations = func.udf(lambda x:remove_punctuations(x),StringType())
news_data = news_data.withColumn('Content', udf_remove_punctuations(func.col('Content')))
news_data = news_data.withColumn('Title', udf_remove_punctuations(func.col('Title')))
news_data.select('Title').show(5, False)

23/11/02 12:18:39 WARN TaskSetManager: Stage 5 contains a task of very large size (19966 KiB). The maximum recommended task size is 1000 KiB.


+------------------------------------------------------------------------------------------+
|Title                                                                                     |
+------------------------------------------------------------------------------------------+
|hawkeye 360 reaches inflection point on path to profitability                             |
|ses q a leveling up multi orbit connectivity                                              |
|rapid starlink iteration poses challenges for resellers                                   |
|space force to release guidelines for the use of commercial satellite services            |
|ula has no issues with space force plan to select three national security launch providers|
+------------------------------------------------------------------------------------------+
only showing top 5 rows



#### Count the words in the field: ‘Title’, and list the most frequent words according to the term frequency in descending order, in total, and per day respectively

In [11]:
#Most frequent words in total for the column Title

news_data.withColumn('word', func.explode(func.split(func.col('Title'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)\
    .show(20, False)

23/11/02 12:18:40 WARN TaskSetManager: Stage 6 contains a task of very large size (19966 KiB). The maximum recommended task size is 1000 KiB.


+----------+-----+
|word      |count|
+----------+-----+
|to        |6356 |
|space     |4701 |
|for       |4372 |
|s         |3648 |
|in        |2491 |
|of        |2462 |
|satellite |2349 |
|launch    |2296 |
|nasa      |2272 |
|on        |2115 |
|the       |1552 |
|force     |1449 |
|u         |1432 |
|and       |1405 |
|with      |1181 |
|a         |1144 |
|new       |1068 |
|satellites|992  |
|spacex    |969  |
|commercial|947  |
+----------+-----+
only showing top 20 rows



In [12]:
#Most frequent words per day in the column Title

news_data.withColumn('word', func.explode(func.split(func.col('Title'), ' ')))\
    .groupBy(func.date_format('Date','yyyy-MM-dd').alias('Date'), 'word')\
    .count()\
    .sort('count', ascending=False)\
    .show(20, False)

23/11/02 12:18:50 WARN TaskSetManager: Stage 8 contains a task of very large size (19966 KiB). The maximum recommended task size is 1000 KiB.


+----------+----------+-----+
|Date      |word      |count|
+----------+----------+-----+
|2011-10-01|space     |50   |
|2011-10-01|to        |40   |
|2011-10-01|s         |38   |
|2011-10-01|the       |27   |
|2011-10-01|in        |27   |
|2011-10-01|nasa      |26   |
|2011-10-01|for       |23   |
|2011-10-01|florida   |20   |
|2011-10-01|on        |18   |
|2011-10-01|u         |18   |
|2011-10-01|satellite |17   |
|2011-10-01|post      |17   |
|2011-10-01|today     |17   |
|2013-12-16|in        |15   |
|2011-10-01|of        |15   |
|2011-10-01|news      |14   |
|2011-10-01|washington|14   |
|2012-12-17|the       |14   |
|2012-12-17|in        |13   |
|2013-12-16|year      |13   |
+----------+----------+-----+
only showing top 20 rows



#### Count the words in the fields: ‘Content’, and list the most frequent words according to the term frequency in descending order, in total, and per day respectively

In [13]:
#Most frequent words in total for the column Content

news_data.withColumn('word', func.explode(func.split(func.col('Content'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)\
    .show(20, False)

23/11/02 12:18:56 WARN TaskSetManager: Stage 10 contains a task of very large size (19966 KiB). The maximum recommended task size is 1000 KiB.


+------+------+
|word  |count |
+------+------+
|the   |753165|
|to    |383449|
|of    |316428|
|and   |279567|
|a     |279116|
|in    |244349|
|that  |164331|
|for   |162311|
|s     |159822|
|is    |133168|
|space |122458|
|said  |121372|
|on    |101829|
|with  |84307 |
|it    |81813 |
|as    |71785 |
|we    |71301 |
|will  |70310 |
|be    |70056 |
|launch|67979 |
+------+------+
only showing top 20 rows



In [14]:
#Most frequent words per day in the column Content

news_data.withColumn('word', func.explode(func.split(func.col('Content'), ' ')))\
    .groupBy(func.date_format('Date','yyyy-MM-dd').alias('Date'), 'word')\
    .count()\
    .sort('Date', ascending=False)\
    .show(20, False)

23/11/02 12:19:05 WARN TaskSetManager: Stage 12 contains a task of very large size (19966 KiB). The maximum recommended task size is 1000 KiB.


+----------+------------+-----+
|Date      |word        |count|
+----------+------------+-----+
|2023-09-14|plan        |1    |
|2023-09-14|valuable    |1    |
|2023-09-14|advanced    |1    |
|2023-09-14|conservative|1    |
|2023-09-14|keeps       |1    |
|2023-09-14|highlighted |1    |
|2023-09-14|private     |2    |
|2023-09-14|2026        |1    |
|2023-09-14|them        |1    |
|2023-09-14|12          |2    |
|2023-09-14|world       |1    |
|2023-09-14|ipo         |2    |
|2023-09-14|could       |1    |
|2023-09-14|good        |1    |
|2023-09-14|layer       |1    |
|2023-09-14|depend      |1    |
|2023-09-14|they        |1    |
|2023-09-14|led         |1    |
|2023-09-14|two         |3    |
|2023-09-14|radio       |2    |
+----------+------------+-----+
only showing top 20 rows



#### Calculate the average number of percentage of published articles in a day, and by authors in a day, respectively

In [15]:
from pyspark.sql.window import Window

# Percentage of published articles in a day

news_data.groupBy(func.date_format('Date','yyyy-MM-dd').alias('Date'))\
    .count()\
    .withColumn('percentage', func.col('count')/19584)\
    .sort('count', ascending=False)\
    .select("Date", "percentage").show()

23/11/02 12:19:29 WARN TaskSetManager: Stage 14 contains a task of very large size (19966 KiB). The maximum recommended task size is 1000 KiB.


+----------+--------------------+
|      Date|          percentage|
+----------+--------------------+
|2011-10-01|  0.0090890522875817|
|2012-09-26|0.001327614379084...|
|2013-09-23|0.001327614379084...|
|2012-10-29|0.001276552287581...|
|2012-11-12|0.001276552287581...|
|2012-10-23|0.001174428104575...|
|2021-08-24|0.001021241830065...|
|2022-04-04|0.001021241830065...|
|2012-09-27|0.001021241830065...|
|2013-12-16|0.001021241830065...|
|2014-09-08|9.701797385620915E-4|
|2012-01-23|9.701797385620915E-4|
|2019-04-08|9.191176470588235E-4|
|2011-05-02|9.191176470588235E-4|
|2011-02-14|9.191176470588235E-4|
|2018-08-08|9.191176470588235E-4|
|2012-12-17|8.680555555555555E-4|
|2011-08-29|8.680555555555555E-4|
|2017-04-05|8.680555555555555E-4|
|2018-04-17|8.680555555555555E-4|
+----------+--------------------+
only showing top 20 rows



In [16]:
"""news_data.groupBy(func.date_format('Date','yyyy-MM-dd').alias('Date'), 'Author')\
    .count()\
    .withColumn('percentage', func.col('count')/19584)\
    .sort('Date', ascending=False)\
    .show()"""

"news_data.groupBy(func.date_format('Date','yyyy-MM-dd').alias('Date'), 'Author')    .count()    .withColumn('percentage', func.col('count')/19584)    .sort('Date', ascending=False)    .show()"

In [17]:
A = news_data.groupBy(func.date_format('Date','yyyy-MM-dd').alias('Date')).agg(func.count("*").alias("Total"))
B = news_data.groupBy(func.date_format('Date','yyyy-MM-dd').alias('Date'), 'Author').count()
C = A.join(B, ['Date']).sort('Total', ascending=False)
C.show()

23/11/02 12:19:33 WARN TaskSetManager: Stage 16 contains a task of very large size (19966 KiB). The maximum recommended task size is 1000 KiB.
23/11/02 12:19:35 WARN TaskSetManager: Stage 18 contains a task of very large size (19966 KiB). The maximum recommended task size is 1000 KiB.


+----------+-----+--------------------+-----+
|      Date|Total|              Author|count|
+----------+-----+--------------------+-----+
|2011-10-01|  178|                  AP|    1|
|2011-10-01|  178|         Amy Klamper|    1|
|2011-10-01|  178|         RIA Novotsi|    1|
|2011-10-01|  178|          SpaceX.com|    1|
|2011-10-01|  178|            CBS News|    2|
|2011-10-01|  178|                 BBC|    1|
|2011-10-01|  178| Las Cruces Sun-News|    2|
|2011-10-01|  178|         RIA Novosti|    8|
|2011-10-01|  178|The Associated Press|    1|
|2011-10-01|  178|AP via Baltimore Sun|    1|
|2011-10-01|  178|The Huntsville Times|    2|
|2011-10-01|  178|Courthouse News S...|    1|
|2011-10-01|  178| The Washington Post|    8|
|2011-10-01|  178|           Roll Call|    1|
|2011-10-01|  178| Winnipeg Free Press|    1|
|2011-10-01|  178|            BBC News|    2|
|2011-10-01|  178|              Xinhua|    4|
|2011-10-01|  178|       Florida Today|   15|
|2011-10-01|  178|              Yo

In [18]:
# Percentage of published articles by Author

C.withColumn('Percentage', func.col('count')/func.col('Total')).select("Date", "Author", "Percentage").show()

23/11/02 12:19:44 WARN TaskSetManager: Stage 20 contains a task of very large size (19966 KiB). The maximum recommended task size is 1000 KiB.
23/11/02 12:19:46 WARN TaskSetManager: Stage 22 contains a task of very large size (19966 KiB). The maximum recommended task size is 1000 KiB.


+----------+--------------------+--------------------+
|      Date|              Author|          Percentage|
+----------+--------------------+--------------------+
|2011-10-01|        Forbes India|0.005617977528089...|
|2011-10-01|   Los Angeles Times|0.011235955056179775|
|2011-10-01| Las Cruces Sun-News|0.011235955056179775|
|2011-10-01| Winnipeg Free Press|0.005617977528089...|
|2011-10-01| The Daily Telegraph|0.005617977528089...|
|2011-10-01|           MSNBC.com|0.005617977528089...|
|2011-10-01|     SpaceRef Canada|0.005617977528089...|
|2011-10-01|               Wired|0.016853932584269662|
|2011-10-01|                 AFP| 0.02247191011235955|
|2011-10-01|            ABC News|0.005617977528089...|
|2011-10-01|         Denise Chow|0.005617977528089...|
|2011-10-01|Los Angeles Daily...|0.005617977528089...|
|2011-10-01|           ITAR-TASS|0.016853932584269662|
|2011-10-01|          SpaceX.com|0.005617977528089...|
|2011-10-01|     The Japan Times|0.005617977528089...|
|2011-10-0